# CAMO-Net ETL – RAW → BRONZE
### Notebook de inspeção passo a passo

Este notebook reproduz o funcionamento do script `01_raw_to_bronze.py`, permitindo inspecionar:

1. Descoberta de arquivos na camada RAW  
2. Testes de leitura com múltiplos encodings  
3. Conversão para parquet  
4. Cálculo de compressão  
5. Adição de metadados de ingestão  
6. Resumo final por arquivo 


In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd

raw_path = Path('../../data/raw/20250101_carga_inicial_ana_roccio/')
bronze_path = Path('../../data/bronze/')

if not raw_path.exists():
    raise FileNotFoundError("Pasta RAW não existe. Ajuste o caminho manualmente.")

In [2]:
csv_files = sorted(raw_path.glob('*.csv'))

print(f"Total de arquivos CSV encontrados: {len(csv_files)}\n")

for i, f in enumerate(csv_files, 1):
    print(f"{i:2d}. {f.name}")


Total de arquivos CSV encontrados: 9

 1. TAB_ATENDIMENTO.csv
 2. TAB_ATENDIMENTO_ANALISE.csv
 3. TAB_ATEND_DIAG.csv
 4. TAB_CIAP_DIAGNOSTICO.csv
 5. TAB_CID_DIAGNOSTICO.csv
 6. TAB_MEDICAMENTO.csv
 7. TAB_MEDPRESCRITO_ANALISE.csv
 8. TAB_MED_PRESCRITO.csv
 9. TAB_UNIDADE_SAUDE.csv


In [3]:
def read_csv_with_fallback(filepath):
    encodings = ['utf-8', 'latin1', 'cp1252']
    separators = [';', ',']
    
    for encoding in encodings:
        for sep in separators:
            try:
                df = pd.read_csv(filepath, encoding=encoding, sep=sep, low_memory=False)
                
                if len(df.columns) > 1:  # pelo menos 2 colunas
                    print(f"  ✓ Lido com encoding={encoding}, sep='{sep}'")
                    return df, encoding
            except Exception:
                continue
    
    raise ValueError(f"Não foi possível ler o arquivo {filepath}")


In [4]:
def process_raw_to_bronze(raw_path, bronze_path):
    bronze_path.mkdir(parents=True, exist_ok=True)
    
    ingestion_timestamp = datetime.now()
    
    csv_files = list(raw_path.glob('*.csv'))
    results = []
    
    for csv_file in csv_files:
        table_name = csv_file.stem
        print("\n" + "─"*80)
        print("Processando:", table_name)
        print("─"*80)
        
        try:
            df, encoding = read_csv_with_fallback(csv_file)
            
            original_rows = len(df)
            original_cols = len(df.columns)
            
            # adicionar metadados
            df['_ingestion_date'] = ingestion_timestamp
            
            # caminho de saída
            output_file = bronze_path / f"{table_name}.parquet"
            
            # salvar parquet
            df.to_parquet(output_file, engine='pyarrow', compression='snappy', index=False)
            
            # tamanhos
            csv_size = csv_file.stat().st_size / (1024 * 1024)
            parquet_size = output_file.stat().st_size / (1024 * 1024)
            compression_ratio = (1 - parquet_size / csv_size) * 100
            
            print(f"  ✓ Salvo: {output_file.name}")
            print(f"  • Linhas: {original_rows:,}")
            print(f"  • Colunas: {original_cols} + 1 metadata")
            print(f"  • CSV: {csv_size:.2f} MB → Parquet: {parquet_size:.2f} MB")
            print(f"  • Compressão: {compression_ratio:.1f}%")
            
            results.append({
                'tabela': table_name,
                'status': 'SUCCESS',
                'linhas': original_rows,
                'colunas': original_cols,
                'encoding': encoding,
                'tamanho_mb': parquet_size
            })
            
        except Exception as e:
            print("  ✗ ERRO:", e)
            results.append({
                'tabela': table_name,
                'status': 'FAILED',
                'erro': str(e)
            })
    
    return results


In [5]:
results = process_raw_to_bronze(raw_path, bronze_path)



────────────────────────────────────────────────────────────────────────────────
Processando: TAB_ATENDIMENTO
────────────────────────────────────────────────────────────────────────────────
  ✓ Lido com encoding=utf-8, sep=','
  ✓ Salvo: TAB_ATENDIMENTO.parquet
  • Linhas: 480,439
  • Colunas: 6 + 1 metadata
  • CSV: 50.67 MB → Parquet: 19.75 MB
  • Compressão: 61.0%

────────────────────────────────────────────────────────────────────────────────
Processando: TAB_MEDICAMENTO
────────────────────────────────────────────────────────────────────────────────
  ✓ Lido com encoding=utf-8, sep=','
  ✓ Salvo: TAB_MEDICAMENTO.parquet
  • Linhas: 33,246
  • Colunas: 11 + 1 metadata
  • CSV: 2.04 MB → Parquet: 0.65 MB
  • Compressão: 68.0%

────────────────────────────────────────────────────────────────────────────────
Processando: TAB_ATENDIMENTO_ANALISE
────────────────────────────────────────────────────────────────────────────────
  ✓ Lido com encoding=utf-8, sep=','
  ✓ Salvo: TAB_ATENDI

In [6]:
df_results = pd.DataFrame(results)
df_results


,tabela,status,linhas,colunas,encoding,tamanho_mb
0,TAB_ATENDIMENTO,SUCCESS,480439,6,utf-8,19.753788
1,TAB_MEDICAMENTO,SUCCESS,33246,11,utf-8,0.654016
2,TAB_ATENDIMENTO_ANALISE,SUCCESS,298848,12,utf-8,10.942658
3,TAB_ATEND_DIAG,SUCCESS,574676,5,utf-8,18.727766
4,TAB_CID_DIAGNOSTICO,SUCCESS,896,8,utf-8,0.038961
5,TAB_UNIDADE_SAUDE,SUCCESS,50,4,utf-8,0.005702
6,TAB_MEDPRESCRITO_ANALISE,SUCCESS,269123,8,utf-8,12.201520
7,TAB_CIAP_DIAGNOSTICO,SUCCESS,726,6,utf-8,0.031170
8,TAB_MED_PRESCRITO,SUCCESS,443454,6,utf-8,13.597899


In [7]:
success = df_results[df_results['status'] == 'SUCCESS']
failed = df_results[df_results['status'] == 'FAILED']

print("Arquivos OK:", len(success))
print("Arquivos com erro:", len(failed))

if len(success) > 0:
    print("\nTotal de linhas processadas:", success['linhas'].sum())
    print("Tamanho total BRONZE:", success['tamanho_mb'].sum(), "MB")


Arquivos OK: 9
Arquivos com erro: 0

Total de linhas processadas: 2101458
Tamanho total BRONZE: 75.95348167419434 MB


In [8]:
generated = sorted(bronze_path.glob('*.parquet'))
generated

[PosixPath('../../data/bronze/TAB_ATENDIMENTO.parquet'),
 PosixPath('../../data/bronze/TAB_ATENDIMENTO_ANALISE.parquet'),
 PosixPath('../../data/bronze/TAB_ATEND_DIAG.parquet'),
 PosixPath('../../data/bronze/TAB_CIAP_DIAGNOSTICO.parquet'),
 PosixPath('../../data/bronze/TAB_CID_DIAGNOSTICO.parquet'),
 PosixPath('../../data/bronze/TAB_MEDICAMENTO.parquet'),
 PosixPath('../../data/bronze/TAB_MEDPRESCRITO_ANALISE.parquet'),
 PosixPath('../../data/bronze/TAB_MED_PRESCRITO.parquet'),
 PosixPath('../../data/bronze/TAB_UNIDADE_SAUDE.parquet')]